# Inference Analysis

### Libraries

In [1]:
import glob
import os
import pandas as pd
import numpy as np
from scipy import stats
import itertools
# import cv2
# import gc
# import time
# import json
# import io
# import logging
import yaml
from tqdm import tqdm
import resource
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from main_inference import main
from ultralytics import YOLO
import supervision as sv
from sklearn.metrics import f1_score,mean_absolute_percentage_error,mean_squared_error


PROJECT_DIR = os.path.join("/", "Users", "aus10powell", "Documents", "Projects", "MIT-Fishery-Counter")
DATA_DIR = os.path.join(PROJECT_DIR, "data")




SITE = "IRWA"

if SITE == "IRWA":
    GOLD_DIR = "/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/data/gold_dataset/videos/irwa"
    assert os.path.exists(GOLD_DIR), "Gold directory does not exist"
mp4_files = glob.glob(os.path.join(GOLD_DIR, "*.mp4"))

assert len(mp4_files) > 0, "No mp4 files found in gold directory"   


### Functions

In [2]:
def calculate_confidence_interval(count_data, confidence_level=0.95):
    # Calculate the mean and standard deviation of the count data
    mean = np.mean(count_data)
    std = np.std(count_data)

    # Calculate the confidence interval using the t-distribution
    n = len(count_data)
    t_value = stats.t.ppf((1 + confidence_level) / 2, df=n-1)
    margin_of_error = t_value * std / np.sqrt(n)
    confidence_interval = (mean - margin_of_error, mean + margin_of_error)

    return confidence_interval

def calculate_mape_and_confidence_interval(y_true, y_pred, n_samples=1000, confidence_level=95):
    """
    Calculate MAPE and its confidence interval using bootstrapping.

    Args:
    - y_true (numpy.ndarray): Array of true values.
    - y_pred (numpy.ndarray): Array of predicted values.
    - n_samples (int): Number of bootstrap samples to generate.
    - confidence_level (int): Confidence level for the interval (e.g., 95 for a 95% confidence interval).

    Returns:
    - tuple: A tuple containing the MAPE value, lower bound, and upper bound of the confidence interval.
    """
    # Initialize an array to store MAPE values
    mape_values = np.zeros(n_samples)

    # Perform bootstrapping
    for i in range(n_samples):
        # Generate a random bootstrap sample with replacement
        bootstrap_indices = np.random.choice(len(y_true), len(y_true), replace=True)
        bootstrap_y_true = y_true[bootstrap_indices]
        bootstrap_y_pred = y_pred[bootstrap_indices]

        # Calculate MAPE for the bootstrap sample
        mape = np.mean(np.abs((bootstrap_y_true - bootstrap_y_pred) / bootstrap_y_true)) * 100

        # Store the MAPE value
        mape_values[i] = mape

    # Calculate the confidence interval bounds
    lower_bound = np.percentile(mape_values, (100 - confidence_level) / 2)
    upper_bound = np.percentile(mape_values, confidence_level + (100 - confidence_level) / 2)

    # Calculate the MAPE value
    mape_value = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return mape_value, lower_bound, upper_bound


def MAPE(y_true, y_pred):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between the true values and predicted values.
    
    Parameters:
        y_true (array-like): Array or list of true values.
        y_pred (array-like): Array or list of predicted values.

    Returns:
        float: The calculated MAPE value.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Avoid division by zero
    epsilon = 1e-10

    total_percentage_error = 0
    total_samples = len(y_true)

    for i in range(total_samples):
        # Calculate the absolute percentage error for each data point
        if y_true[i] == 0:
            absolute_percentage_error = np.abs((y_true[i] - y_pred[i]))
        else:
            absolute_percentage_error = np.abs((y_true[i] - y_pred[i]) / (y_true[i] + epsilon))

        # Add it to the total percentage error
        total_percentage_error += absolute_percentage_error

    # Calculate the mean of the absolute percentage errors
    mape = (total_percentage_error / total_samples) * 100.0

    return mape

# Define a function to read and update the YAML file
def update_yaml_file(file_path, new_values):
    try:
        # Read the existing YAML file
        with open(file_path, 'r') as yaml_file:
            data = yaml.safe_load(yaml_file)
        

        # Update the values or add new ones
        for key, value in new_values.items():
            data[key] = value

        # # Write the updated data back to the YAML file
        with open(file_path, 'w') as yaml_file:
            yaml.dump(data, yaml_file, default_flow_style=False)
        
        print("YAML file updated successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")

def process_video_data(data, mp4_files, tracker=None):
    """
    Process video data and return a DataFrame with results.

    Args:
    - data (list): List of data entries to process.
    - mp4_files (list): List of MP4 file paths.

    Returns:
    - pd.DataFrame: DataFrame with processed results.
    """

    # Initialize variables
    pred_net_counts = []  # List to store predicted net counts
    true_net_counts = []  # List to store true net counts
    total_duration_seconds = []  # List to store total duration in seconds
    videos_missed = []  # List to store information about missed videos

    # Loop through each data entry using tqdm for progress bar
    print(f"num files: {len(data)}")
    for idx in tqdm(range(len(data))):
        # Find the video path that matches the current data entry
        video_path = [path for path in mp4_files if data[idx]["file"] in path][0]
        #print(video_path)

        # Call the main function with video_path and other parameters
        frame_rate, annotated_frames, out_count, in_count, duration_seconds, _ = main(
            video_path=video_path,
            device='mps',
            tracker=tracker,
            stream=False,
            show=False
        )
        print("*"*1000)
        # Calculate the predicted net counts and append to the list
        pred_net_counts.append(out_count - in_count)

        # Check if the predicted net counts differ from the true counts
        if (out_count - in_count) != data[idx]["true_herring_counts"]:
            videos_missed.append({'file': data[idx], 'pred': out_count - in_count, 'true': data[idx]["true_herring_counts"]})

        # Append true net counts and total duration to their respective lists
        true_net_counts.append(data[idx]["true_herring_counts"])
        total_duration_seconds.append(duration_seconds)

    # Create a DataFrame to store the results
    df_results = pd.DataFrame(data={"pred_net_counts": pred_net_counts, "true_net_counts": true_net_counts})

    return df_results

def calculate_metrics(df_results):
    # Create an empty DataFrame to store metrics
    df_metrics = pd.DataFrame()

    # Calculate the number of videos with missed predictions
    videos_missed = df_results[df_results["pred_net_counts"] != df_results["true_net_counts"]].shape[0]

    # Calculate the total true net counts
    total_true_count = df_results["true_net_counts"].sum()

    # Extract prediction and true net counts
    pred_net_counts = df_results["pred_net_counts"]
    true_net_counts = df_results["true_net_counts"]

    # Calculate the absolute difference between true and predicted counts for each row
    true_minus_pred = df_results.apply(lambda r: r["true_net_counts"] - r["pred_net_counts"], axis=1)

    # Calculate Root Mean Squared Error (RMSE)
    rmse = mean_squared_error(y_pred=pred_net_counts, y_true=true_net_counts)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_pred=pred_net_counts, y_true=true_net_counts)

    # Calculate Mean Absolute Percentage Error (MAPE)
    mape = mean_absolute_percentage_error(true_net_counts, pred_net_counts)

    # Calculate F1 Score
    f1 = f1_score(y_true=df_results["true_net_counts"], y_pred=df_results["pred_net_counts"], average="macro")

    # Calculate total misscounts
    misscounts = sum(true_minus_pred)

    # Calculate the total true net counts (again, for clarity)
    total_herring = total_true_count

    # Calculate the total percent error
    total_percent_error = misscounts / total_herring

    # Populate the DataFrame with calculated metrics
    df_metrics["videos_missed"] = [videos_missed]
    df_metrics["mape"] = [mape]
    df_metrics["total_percent_error"] = [total_percent_error]
    df_metrics["misscounts"] = [misscounts]
    df_metrics["total_herring"] = [total_herring]
    df_metrics["rmse"] = [rmse]
    df_metrics["mae"] = [mae]
    df_metrics["f1"] = [f1]

    return df_metrics

## Inference 

### Scoring on gold standard

In [4]:
%%capture

# True data
data_2018 = [
    {"file": "2_2018-05-10_06-39-30", "true_herring_counts": 4},
    {"file": "2_2018-04-14_10-06-19","true_herring_counts": 1 },
    {"file": "2_2018-04-14_13-18-51", "true_herring_counts": 1},
    {"file": "2_2018-04-28_10-54-38", "true_herring_counts": 3},
    {"file": "2_2017-06-04_06-09-56", "true_herring_counts": 0}, # Comes from left returns right
    {"file": "2_2017-04-15_11-23-36", "true_herring_counts": 1},
    {"file": "2_2017-04-13_14-10-29", "true_herring_counts": 1},  # 2_2017-04-13_13-10-00
    {"file": "2_2017-04-13_13-10-00", "true_herring_counts": 1}, # 2_2018-04-14_17-12-42
    {"file": "2_2018-04-14_17-12-42", "true_herring_counts": 1}, # 2_2018-04-27_13-07-38
    {"file": "2_2018-04-27_13-07-38", "true_herring_counts": 1}, # 2_2018-04-27_15-23-03
    {"file": "2_2018-04-27_15-23-03", "true_herring_counts": 3}, 
    {"file": "2_2018-04-29_08-28-10", "true_herring_counts": 1},
    {"file": "2_2018-04-29_09-14-03", "true_herring_counts": 1},
    {"file": "2_2018-04-29_16-28-35", "true_herring_counts": 2},
    {"file": "2_2018-04-29_15-55-24", "true_herring_counts": 2},
    {"file": "2_2018-04-28_11-25-56", "true_herring_counts": 2},
    {"file": "2_2018-04-29_15-39-37", "true_herring_counts": 3},
    {"file": "2_2018-04-29_16-54-05", "true_herring_counts": 1}, 
    {"file": "2_2018-05-04_11-32-10", "true_herring_counts": 1}, 
    {"file": "2_2018-05-22_05-58-08", "true_herring_counts": 2}, 
    {"file": "2_2018-05-04_09-24-42", "true_herring_counts": 2}, # 2_2018-05-04_09-24-42 
    {"file": "2_2018-05-05_10-50-59", "true_herring_counts": 2}, # 2_2018-05-05_10-50-59
    {"file": "2_2018-05-05_13-31-05", "true_herring_counts": 1}, # 2_2018-05-05_13-31-05
    {"file": "2_2018-05-04_18-57-00", "true_herring_counts": 3}, # 2_2018-05-04_18-57-00
    {"file": "2_2018-05-05_13-49-11", "true_herring_counts": 1}, # 2_2018-05-05_13-49-11
    {"file": "2_2018-05-05_18-10-49", "true_herring_counts": 1}, # 2_2018-05-05_18-10-49
    {"file": "2_2018-05-05_18-47-03", "true_herring_counts": 2}, # 2_2018-05-05_18-47-03
    {"file": "2_2018-05-06_08-25-36", "true_herring_counts": 2}, # 2_2018-05-06_08-25-36
    {"file": "2_2018-04-27_12-01-34", "true_herring_counts": 1},#  
    {"file": "2_2018-04-29_18-31-06", "true_herring_counts": 1},# 
    {"file": "2_2018-04-29_17-33-32", "true_herring_counts": 2}, # 
    {"file": "2_2018-05-02_10-09-38", "true_herring_counts": 2},# 
    {"file": "2_2018-05-06_17-32-24", "true_herring_counts": 1}, # 2_2018-05-06_16-35-26
    {"file": "2_2018-05-06_16-35-26", "true_herring_counts": 2}, # 
    {"file": "2_2018-05-06_17-08-27", "true_herring_counts": 1}, # 
    {"file": "2_2018-05-06_18-09-56", "true_herring_counts": 3},# 
    {"file": "2_2018-04-29_16-17-14", "true_herring_counts": 1}, # 
    {"file": "2_2018-05-13_11-44-28", "true_herring_counts": 1}, # 
    {"file": "2_2018-05-13_07-46-27", "true_herring_counts": 1}, # 
    {"file": "2_2018-05-11_19-06-07", "true_herring_counts": 2}, # 
    {"file": "2_2018-05-10_17-43-39", "true_herring_counts": 1}, # 
    {"file": "2_2018-05-10_17-50-04", "true_herring_counts": 2}, # 
    {"file": "2_2018-05-05_13-15-07", "true_herring_counts": 1}, # 
    {"file": "2_2018-05-05_13-00-52", "true_herring_counts": 2},#  
    {"file": "2_2018-05-05_13-36-08", "true_herring_counts": 2}, # 
    {"file": "2_2018-05-05_12-04-58", "true_herring_counts": 3},# 
]

data_2017 = [{"file":"2_2017-04-29_09-09-49","true_herring_counts": 1},
{"file":"2_2017-04-29_11-17-59","true_herring_counts": 1},
{"file":"2_2017-04-29_17-32-56","true_herring_counts": 1},
{"file":"2_2017-04-29_16-08-35","true_herring_counts": 1},
{"file":"2_2017-04-29_17-37-45","true_herring_counts": 3},
{"file":"2_2017-04-29_17-43-47","true_herring_counts": 1},  
{"file":"2_2017-04-29_17-59-00","true_herring_counts": 2},   #  2
{"file":"2_2017-04-30_07-36-18","true_herring_counts": 2}, # 2
{"file":"2_2017-05-04_15-17-51","true_herring_counts": 1}, # 2_2017-05-04_15-17-51 1
{"file":"2_2017-04-29_18-28-48","true_herring_counts": 2}, # 2_2017-04-29_18-28-48 2
{"file":"2_2017-04-30_08-59-51","true_herring_counts": 2}, 
{"file":"2_2017-04-29_18-32-08","true_herring_counts": 2}, 
{"file":"2_2017-04-29_15-58-43","true_herring_counts": 1}, 
{"file":"2_2017-05-19_08-40-04","true_herring_counts": 2}, 
{"file":"2_2017-05-19_09-48-27","true_herring_counts": 2}, 
{"file":"2_2017-04-13_11-40-43","true_herring_counts": 1}, 
]

data_2016 = [{"file":"1_2016-04-13_13-57-11","true_herring_counts":1},
{"file":"1_2016-04-13_13-57-11","true_herring_counts":5}, 
{"file":"1_2016-04-22_12-36-58","true_herring_counts":2}, 
{"file":"1_2016-04-24_15-38-03","true_herring_counts":3},

{"file":"1_2016-04-22_12-33-05","true_herring_counts":1},
{"file":"1_2016-04-22_16-50-00","true_herring_counts":3},
{"file":"1_2016-04-30_11-50-35","true_herring_counts":1},
{"file":"1_2016-05-09_18-59-02","true_herring_counts":1},
{"file":"1_2016-05-09_12-31-06","true_herring_counts":1},
{"file":"1_2016-04-23_16-30-41","true_herring_counts":3},
]

df_results = process_video_data(data=data_2018,mp4_files=mp4_files,tracker="/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/src/utils/tracking_configs/botsort.yaml") # "../botsort_manipulate.yaml"
display(calculate_metrics(df_results=df_results))



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/212) /Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/data/gold_dataset/videos/irwa/1_2016-04-13_13-57-11.mp4: 256x320 (no detections), 311.8ms
video 1/1 (frame 2/212) /Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/data/gold_dataset/videos/irwa/1_2016-04-13_13-57-11.mp4: 256x320 (no detections), 22.1ms
video 1/1 (frame 3/212) /Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/data/gold_dataset/videos/irwa/1

ValueError: too many values to unpack (expected 5)

In [ ]:
mp4_files

In [ ]:
display(calculate_metrics(df_results=df_results))

In [ ]:
display(calculate_metrics(df_results=df_results))

### Grid Search for Tracking Algorithm

In [ ]:
"""
Initialize parameters
"""
fpath = "../botsort_manipulate.yaml"

parameters = {
    "track_high_thresh": np.linspace(start=0.5, stop=.9, num=2),    # Values from 0.0 to 0.8 in steps of 0.1
    "track_low_thresh": np.linspace(start=0.1, stop=.5, num=3),     # Values from 0.0 to 0.8 in steps of 0.1
    "new_track_thresh": np.linspace(start=0.1, stop=.6, num=2),     # Values from 0.0 to 0.8 in steps of 0.1
    "match_thresh": np.linspace(start=0.1, stop=.99, num=3),         # Values from 0.0 to 0.8 in steps of 0.1
    "proximity_thresh": np.linspace(start=0.2, stop=.99, num=1)      # Values from 0.0 to 0.8 in steps of 0.1
}


# tracker_type: botsort  # tracker type, ['botsort', 'bytetrack']
# track_high_thresh: 0.8   # 0.6 default threshold for the first association
# track_low_thresh: 0.4  # threshold for the second association
# new_track_thresh: 0.2  # 0.6 default threshold for init new track if the detection does not match any tracks
# track_buffer: 30  # buffer to calculate the time when to remove tracks
# match_thresh: 0.90  # 0.8 default threshold for matching tracks
# # min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)
# # mot20: False  # for tracker evaluation(not used for now)

# # BoT-SORT settings
# cmc_method: sparseOptFlow  # method of global motion compensation
# # ReID model related thresh (not supported yet)
# proximity_thresh: 0.8 # 0.2 default
# appearance_thresh: 0.1
# with_reid: False

grid = itertools.product(*parameters.values())

print("*"*50)
grid = list(grid)
num_parameters = len(grid)
print(f"Estimated completion time: {(64/9) * num_parameters}")
print(f"Searching across {num_parameters} parameters")

In [ ]:
%%capture

df_metrics = pd.DataFrame()
# Perform grid search
for combo in tqdm(grid): # itertools.product(*parameters.values())
    track_high_thresh, track_low_thresh, new_track_thresh, match_thresh, proximity_thresh = combo
    
    new_values = {"track_high_thresh":float(track_high_thresh), 
                "track_low_thresh":float(track_low_thresh), 
                "new_track_thresh":float(new_track_thresh),  
                "match_thresh": float(match_thresh), 
                "proximity_thresh":float(proximity_thresh)}
    print("new_values",new_values)

    # Process video with new parameters
    df_results = process_video_data(data=data_2016,mp4_files=mp4_files,tracker="../botsort_manipulate.yaml") # "../botsort_manipulate.yaml"
    # Calculate metrics for results
    df_metric = calculate_metrics(df_results=df_results)
    df_metric["track_high_thresh"] = track_high_thresh
    df_metric["track_low_thresh"] = track_low_thresh 
    df_metric["new_track_thresh"] = new_track_thresh
    df_metric["match_thresh"] = match_thresh
    df_metric["proximity_thresh"] =proximity_thresh 

    # Append to overal results
    df_metrics = pd.concat([df_metrics,df_metric])
    print(df_metric)
    # Update tracking parameters
    update_yaml_file(file_path=fpath,new_values=new_values)
    gc.collect()


In [ ]:


# print("num videos predicted wrong: ",videos_missed)
# print(f"Total number of videos process: {len(df_results)}")

# print(f"MAE (Mean Absolute Error e.g. counts): {mae:.2f}")
# print(f"RMSE: {rmse}")
# print(f"MAPE (Mean Absolute Percentage Error of Counts): {mape:.2%}",)
# print(f"Overall Percent Error {gmape:.2%}")
# print(f"Did not count {misscounts} out of {total_true_count} herring")
# print(f"f1 macro: {f1}")

df_metrics = calculate_metrics(df_results=df_results)
display(df_metrics)



# absolute_error = np.abs(df_results["pred_net_counts"] - df_results["true_net_counts"]).values
# std_absolute_error = np.std(absolute_error)
# ci = calculate_confidence_interval(count_data=diffs)
# print(f"Estimates off between {ci[0]:.1f} and {ci[1]:.1f} fish at an average of {np.mean(diffs):.1f} per video")


In [ ]:
calculate_mape_and_confidence_interval(y_pred=pred_net_counts,y_true = true_net_counts)

In [ ]:
from roboflow import Roboflow
from pprint import pprint

rf = Roboflow(api_key="7SDCfXc1vaECZpcEb0TR")
ws = rf.workspace()#.project("coonamesset v1")
print(ws.projects())
project = rf.project("coonamesset/coonamesset-v1")
print(project.versions())
version = project.version()

# project.deploy("model-type", "path/to/training/results/", "weights_filename")
pprint(dir(project))

In [ ]:
dir(project)[-10:]

In [ ]:
!ls

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

# Define number of rows and columns in the grid
num_rows = 4
num_cols = 5

# Define class labels (balanced distribution)
class_labels = ["herring", "dog"] 
class_labels = class_labels * (num_rows * num_cols // len(class_labels))

# Randomize the order of class labels
random.shuffle(class_labels)

# Define rectangle width and height
rect_width = 0.3
rect_height = 0.2

# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Set spacing between rectangles
x_step = (1 - num_cols * rect_width) / (num_cols + 1)
y_step = (1 - num_rows * rect_height) / (num_rows + 1)

# Starting position for the first rectangle
x_start = 0.05
y_start = 0.9

# Plot rectangles with corresponding class labels
for i in range(num_rows):
    for j in range(num_cols):
        x_min = x_start + j * (rect_width + x_step)
        y_min = y_start - i * (rect_height + y_step)
        rect = patches.Rectangle((x_min, y_min), rect_width, rect_height, color='gray', alpha=0.3)
        #ax.add_patch(rect)
        
        # Randomly assign position for the label within the rectangle
        label_x = random.uniform(x_min + 0.05, x_min + rect_width - 0.05)
        label_y = random.uniform(y_min + 0.05, y_min + rect_height - 0.05)
        ax.text(label_x, label_y, class_labels[i * num_cols + j], ha='center', va='center')

# Set axis limits and labels
ax.set_xlim(0, 1.2)
ax.set_ylim(0, 1.2)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('X')
ax.set_ylabel('Y')

# Add title
ax.set_title('Balanced Object Detection Dataset (Example)')

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
"""

 "/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/notebooks/runs/colab_runs/last3.pt" # best12.pt
num videos predicted wrong:  9
Total video processed: 598.044 (sec), 10.0 (min)
Total number of videos process: 46
Did not count 11 out of 75 herring
MAE (Mean Absolute Error e.g. counts): 0.24
MAPE (Mean Absolute Percentage Error of Counts): 11.6%
Average number of missed counts per second 0.02
Estimates off between 0.1 and 0.4 fish at an average of 0.2 per video


"/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/notebooks/runs/colab_runs/last2.pt" (medium model)
num videos predicted wrong:  9
Total video processed: 598.044 (sec), 10.0 (min)
Total number of videos process: 46
Did not count 11 out of 75 herring
MAE (Mean Absolute Error e.g. counts): 0.24
MAPE (Mean Absolute Percentage Error of Counts): 13.0%
Average number of missed counts per second 0.02
Estimates off between 0.1 and 0.4 fish at an average of 0.2 per video

"/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/notebooks/runs/colab_runs/best5.pt"
Total video processed: 370.959 (sec), 6.2 (min)
Total number of videos process: 28
Did not count 8 out of 46 herring
MAE (Mean Absolute Error e.g. counts): 0.29
MAPE (Mean Absolute Percentage Error of Counts): 15.5%
Average number of missed counts per second 0.02
Estimates off between 0.1 and 0.5 fish at an average of 0.3 per video


"/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/notebooks/runs/colab_runs/best3.pt"

Total video processed: 347.549 (sec), 5.8 (min)
Total number of videos process: 26
Did not count 7 out of 42 herring
MAE (Mean Absolute Error e.g. counts): 0.27
MAPE (Mean Absolute Percentage Error of Counts): 16.7%
Average number of missed counts per second 0.02
Estimates off between 0.0 and 0.5 fish at an average of 0.3 per video


 "/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/notebooks/runs/colab_runs/best2.pt"
 
Total video processed: 311.58 (sec), 5.2 (min)
Total number of videos process: 23
Did not count 6 out of 37 herring
MAE (Mean Absolute Error e.g. counts): 0.26
MAPE (Mean Absolute Percentage Error of Counts): 17.4%
Average number of missed counts per second 0.02
Estimates off between -0.0 and 0.5 fish at an average of 0.3 per video


/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/runs/detect/train79/weights/last.pt

Total video processed: 311.58 (sec), 5.2 (min)
Total number of videos process: 23
Did not count 10 out of 37 herring
MAE (Mean Absolute Error e.g. counts): 0.43
MAPE (Mean Absolute Percentage Error of Counts): 21.0%
Average number of missed counts per second 0.03
Estimates off between 0.2 and 0.7 fish at an average of 0.4 per video


# /Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/notebooks/runs/detect/train196/weights/last.pt

Total video processed: 284.347 (sec), 4.7 (min)
Total number of videos process: 21
Did not count 14 out of 34 herring
MAE (Mean Absolute Error e.g. counts): 0.67
MAPE (Mean Absolute Percentage Error of Counts): 38.1%
Average number of missed counts per second 0.05
Estimates off between 0.3 and 1.0 fish at an average of 0.7 per video
"""

## Estimating on All Historical Data

In [ ]:
def extract_file_datetime(fname):
    """Extract datetime from file name
    
    Args:
        fname (str): File name

    Returns:
        pd.datetime: Datetime extracted from file name  
    """
    fname = os.path.basename(fname)
    dt = fname.split("_")[1]
    h,m,s = fname.split("_")[2].split(".")[0].split("-")
    return pd.to_datetime(f"{dt} {h}:{m}:{s}")

In [ ]:
import glob, os
HIST_DATA_DIR= "/Users/aus10powell/Downloads/RiverHerring/IRWA 2017 Videos (COMPLETE)"

all_2015 = glob.glob(os.path.join(HIST_DATA_DIR, "2015 Fish sightings", "*.mp4"))
print(f"Num videos in 2015: {len(all_2015)}")

all_2016 = glob.glob(os.path.join(HIST_DATA_DIR, "Fish Sightings 2016", "*.mp4"))
print(f"Num videos in 2016: {len(all_2016)}")

all_2017 = glob.glob(os.path.join(HIST_DATA_DIR, "Fish Sightings 2017", "*.mp4"))
print(f"Num videos in 2017: {len(all_2017)}")

all_2018 = glob.glob(os.path.join(HIST_DATA_DIR, "2018 Fish Sightings", "*.mp4"))
print(f"Num videos in 2018: {len(all_2018)}")

In [ ]:
%%capture

import time
import gc

data = all_2015

# Initialize variables
pred_net_counts = []  # List to store predicted net counts
processing_duration_seconds = []  # List to store total duration in seconds
start_dts = []
end_dts = []

batch_size = 10  # Number of data entries to process in each batch

# Loop through data in batches
for batch_start in range(0, len(data), batch_size):
    batch_end = batch_start + batch_size
    batch_data = data[batch_start:batch_end]

    # Process data in the current batch
    for idx in tqdm(range(len(batch_data))):
        t0 = time.time()
        # Call the main function with video_path and other parameters
        frame_rate, annotated_frames, out_count, in_count, duration_seconds, _ = main(
            video_path=batch_data[idx],
            device='mps',
            tracker="/Users/aus10powell/Documents/Projects/MIT-Fishery-Counter/code/src/utils/tracking_configs/botsort.yaml",
            stream=True,
            show=False
        )
        start_dt = extract_file_datetime(batch_data[idx])
        end_dt = start_dt + pd.Timedelta(seconds=float(duration_seconds), unit='s')

        # Calculate the predicted net counts and append to the list
        pred_net_counts.append(out_count - in_count)
        start_dts.append(start_dt)
        end_dts.append(end_dt)
    
    # Clear memory after each batch
    del batch_data
    gc.collect()

    

In [ ]:
df_results = pd.DataFrame(data={"pred_net_counts": pred_net_counts, "start_dt": start_dts, "end_dt": end_dts})
df_results["duration_seconds"] = df_results["end_dt"] - df_results["start_dt"]
year = int(max(df_results["start_dt"]).year)
fpath = f"inference_data/{year}_df_results.csv"
df_results.to_csv(os.path.join(DATA_DIR,fpath),index=False)
print(df_results.shape)
display(df_results.head(2))


### Analysis on Historical Inference

#### Visualizations

In [ ]:
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def plot_fish_counters(data_frame):
#     """
#     Plots fish counters data over time, including total counts and a 7-day rolling average.

#     Args:
#         data_frame (pd.DataFrame): DataFrame containing fish counters data with columns 'start_dt' and 'pred_net_counts'.

#     Returns:
#         go.Figure: Plotly figure displaying total counts and 7-day rolling average.
#     """
#     # Extract date and hour of day from start_dt
#     data_frame['date'] = data_frame['start_dt'].dt.date
#     data_frame['hour'] = data_frame['start_dt'].dt.hour

#     # Group by date and sum the counts
#     grouped_df = data_frame.groupby([pd.Grouper(key='start_dt', freq='D')])['pred_net_counts'].sum().reset_index()

#     # Calculate rolling average with a window size of 7 days
#     grouped_df['rolling_average'] = grouped_df['pred_net_counts'].rolling(window=7).mean()

#     year = int(max(data_frame["start_dt"]).year)
#     total = data_frame["pred_net_counts"].sum()

#     # Create traces for bar chart and line chart
#     bar_trace = go.Bar(
#         x=grouped_df['start_dt'],
#         y=grouped_df['pred_net_counts'],
#         name='Total Counts'
#     )

#     line_trace = go.Scatter(
#         x=grouped_df['start_dt'],
#         y=grouped_df['rolling_average'],
#         mode='lines',
#         name='7-day Rolling Average'
#     )

#     # Create figure and add traces
#     fig = go.Figure(data=[bar_trace, line_trace])

#     # Set the line color to red and show the legend
#     fig.update_traces(selector=dict(type='scatter'), line_color='red', showlegend=True)
#     fig.update_layout(
#         xaxis_title='Date',
#         yaxis_title='Count',
#         title=f'Fish Counters over time for {year} (total counts {total})',
#         width=800,
#         height=400
#     )

#     return fig

# plot_fish_counters(data_frame=df_results)

In [ ]:
import pandas as pd
import altair as alt

import pandas as pd
import altair as alt

def plot_fish_counters(data_frame):
    """
    Plots fish counters data over time, including total counts and a 7-day rolling average.

    Args:
        data_frame (pd.DataFrame): DataFrame containing fish counters data with columns 'start_dt' and 'pred_net_counts'.

    Returns:
        alt.Chart: Altair chart displaying total counts and 7-day rolling average.
    """
    # Group by date and sum the counts
    grouped_df = data_frame.groupby([pd.Grouper(key='start_dt', freq='D')])['pred_net_counts'].sum().reset_index()

    # Calculate rolling average with a window size of 7 days
    grouped_df['rolling_average'] = grouped_df['pred_net_counts'].rolling(window=7).mean()

    year = int(max(data_frame["start_dt"]).year)
    total = data_frame["pred_net_counts"].sum()

    # Create Altair chart
    bar_chart = alt.Chart(grouped_df).mark_bar().encode(
        x='start_dt:T',
        y='pred_net_counts:Q',
        color=alt.value('steelblue'),
        tooltip=['start_dt:T', 'pred_net_counts:Q']
    ).properties(
        width=800,
        height=400
    ).interactive()

    line_chart = alt.Chart(grouped_df).mark_line(color='red').encode(
        x='start_dt:T',
        y='rolling_average:Q'
    )

    # Combine bar and line charts
    chart = bar_chart + line_chart

    return chart

# Example usage:
# chart = plot_fish_counters(data_frame=df_results)
# chart.save('fish_counters_chart.html')


# Usage
plot_fish_counters(data_frame=df_results)


#### Viewing Summary Read from Local File

In [ ]:
year=2018
fpath = f"inference_data/{year}_df_results.csv"

df_results = pd.read_csv(os.path.join(DATA_DIR,fpath))
df_results["start_dt"] = pd.to_datetime(df_results["start_dt"]) 
display(f"{year}")
display(df_results["start_dt"].describe())
display(df_results["pred_net_counts"].describe())
display("Sum total",df_results["pred_net_counts"].sum())


#### Inference Analysis on all years

In [ ]:
df_results = pd.DataFrame(columns=["pred_net_counts","start_dt","end_dt","duration_seconds"])
for year in [2015,2016,2017,2018]:
    df_temp = pd.read_csv(os.path.join(DATA_DIR,f"inference_data/{year}_df_results.csv"))
    df_results = pd.concat([df_results,df_temp])
df_results["start_dt"] = pd.to_datetime(df_results["start_dt"])
df_results["end_dt"] = pd.to_datetime(df_results["end_dt"])
display("Yearly sums:",df_results.groupby(df_results["start_dt"].dt.year)["pred_net_counts"].sum())
plot_fish_counters(data_frame=df_results)

#### Manually Counted Analysis

In [ ]:

hists = os.listdir("../../data/historical_csv_files/")

df_historical=pd.DataFrame()
for csv_file in hists:
    try:
        df_temp = pd.read_csv(f"../../data/historical_csv_files/{csv_file}")
    except:
        continue
    df_historical = pd.concat([df_historical,df_temp])
df_historical["Date"] = pd.to_datetime(df_historical["Date"])
print("*"*50)
print("Historical Manual Counts for 2015-2018")
display(df_historical.groupby(df_historical["Date"].dt.year)["Count"].sum())

#### Historical vs Predicted Analysis

In [ ]:
year = 2018
df_historical["Date"] = pd.to_datetime(df_historical["Date"])
df_temp  = df_historical[df_historical["Date"].dt.year==year]

df_2015 = pd.read_csv(os.path.join(DATA_DIR,f"inference_data/{year}_df_results.csv"))
df_2015["start_dt"] = pd.to_datetime(df_2015["start_dt"])
df_2015["end_dt"] = pd.to_datetime(df_2015["end_dt"])
df_2015["Date"] = pd.to_datetime(df_2015["start_dt"].dt.date)

min_date = df_temp['Date'].min()
max_date = df_temp['Date'].max()
complete_date_range = pd.date_range(start=min_date, end=max_date, freq='D')
missing_dates = complete_date_range[~complete_date_range.isin(df_temp['Date'])]

print("Missing Dates in manually annotated:")
print(missing_dates)

complete_video_date_range = pd.date_range(start=df_2015["Date"].min(), end=df_2015["Date"].max(), freq='D')
missing_video_dates = complete_video_date_range[~complete_video_date_range.isin(df_2015['Date'])]
print("Missing Dates in videos:")
print(missing_video_dates)
print("*"*50)


common_dates = pd.merge(df_2015, df_temp, on="Date", how='inner')
common_dates = common_dates["Date"]

print("Common Dates:")
print(common_dates)
print(f"{len(set(common_dates))} out of {len(set(complete_date_range))} dates in {year} are common between historical and predicted counts")

In [ ]:
df_historical[df_historical["Date"].dt.year==year]

In [ ]:
display(df_historical)
df_historical["Time"] = pd.to_datetime(df_historical["Time"], errors='coerce')

df_historical["Time"] = df_historical["Time"].dt.strftime('%H:%M:%S')
def combine_and_convert(row):
    combined_string = str(row["Date"].date())+ " " + str(row["Time"])
    return pd.to_datetime(combined_string, format="%Y-%m-%d %H:%M:%S",errors='coerce')

# Apply the function to create a new column "CombinedDatetime"
df_historical["dt_time"] = df_historical.apply(combine_and_convert, axis=1)

start_dt,end_dt,predicted,actual,hist_dates  = [],[],[],[],[]
for t,row in df_historical[df_historical["Date"].dt.year==year].iterrows():
        for t2, row2 in df_2015.iterrows():
              if  row2["start_dt"] <= row["dt_time"] <= row2["end_dt"]:
                    start_dt.append(row2["start_dt"])
                    end_dt.append(row2["end_dt"])
                    predicted.append(row2["pred_net_counts"])
                    actual.append(row["Count"])
                    hist_dates.append(row["dt_time"])
df_historical_vs_predicted = pd.DataFrame(data={"start_dt":start_dt,"end_dt":end_dt,"predicted":predicted,"actual":actual,"hist_dates":hist_dates})
df_historical_vs_predicted.dt.date

In [ ]:
import seaborn as sns
# 2018-04-29
df_temp = df_historical_vs_predicted#[df_historical_vs_predicted["hist_dates"].dt.date.astype(str) == '2018-04-29']
df_temp = df_temp[~df_temp["true_net_counts"].isna()]
df_temp.rename(columns={"actual":"true_net_counts","predicted":"pred_net_counts"},inplace=True)
display(calculate_metrics(df_results=df_temp))
#sns.barplot(data=df_temp,x="true_net_counts",y="pred_net_counts")
df_temp.groupby(df_temp["hist_dates"].dt.date)["true_net_counts"].sum().sort_values()